# <center>Laboratorium Analiza i bazy danych </center>

## <center>TIER protocol i tidy data</center>

Aleksandra Stachniak, grupa 1

In [10]:
import pandas as pd
import os

p = "../"

df = pd.read_csv(p + "original_data/earthquake_data.csv")
df.head(10)

<<<<<<< HEAD
In general, how worried are you about earthquakes? How worried are you about the Big One, a massiv... Do you think the "Big One" will occur in your l... Have you ever experienced an earthquake? Have you or anyone in your household taken any ... How familiar are you with the San Andreas Fault... How familiar are you with the Yellowstone Super... Age     What is your gender? How much total combined money did all members o...           US Region
Not at all worried                                 Not so worried                                     No                                                 Yes, one or more minor ones              No                                                 Somewhat familiar                                  Not so familiar                                    18 - 29 Male                 Prefer not to answer                                       New England
Somewhat worried                                   Very worried                                       No                                                 Yes, one or more minor ones              No                                                 Not at all familiar                                Not at all familiar                                18 - 29 Male                 $75,000 to $99,999                                  East North Central
Not so worried                                     Somewhat worried                                   No                                                 Yes, one or more minor ones              No                                                 Very familiar                                      Somewhat familiar                                  18 - 29 Male                 $10,000 to $24,999                                             Pacific
                                                   Not so worried                                     No                                                 Yes, one or more minor ones              No                                                 Very familiar                                      Not so familiar                                    18 - 29 Male                 $25,000 to $49,999                                  West South Central
                                                                                                      Yes                                                Yes, one or more minor ones              No                                                 Somewhat familiar                                  Extremely familiar                                 18 - 29 Male                 $200,000 and up                                        Middle Atlantic
Not at all worried                                 Not at all worried                                 No                                                 Yes, one or more minor ones              No                                                 Very familiar                                      Not so familiar                                    18 - 29 Male                 $25,000 to $49,999                                  East North Central
Very worried                                       Not at all worried                                 Yes                                                Yes, one or more major ones              Yes                                                NaN                                                NaN                                                NaN     NaN                  NaN                                                                NaN
Not at all worried                                 Not at all worried                                 No                                                 No                                       No                                                 Not so familiar                                    Not at all familiar                                18 - 29 Male                 Prefer not to answer               